# 🤖 Anthropic Claude SDK — Agentic Workflows Tutorial

This notebook walks you through building **agentic workflows** with the **Anthropic Python SDK** step by step:

| Section | What you'll learn |
|---|---|
| **1** | Installation & setup |
| **2** | Your first Claude message |
| **3** | Function calling (tool use) — manual loop |
| **4** | Function calling with `@beta_tool` + tool runner (automatic loop) |
| **5** | Multi-agent system — routing & handoffs |
| **6** | Full practical example — customer support system |
| **7** | Inspecting results & conversation history |
| **8** | Proposed project structure for production |

---

### Key concepts

The Anthropic SDK uses a straightforward **Messages API** with tool use:

- **Messages API** — Send messages to Claude and get responses. The core building block.
- **Tools** — Define tools with a name, description, and JSON schema. Claude decides when to call them.
- **Tool use loop** — When Claude wants to call a tool, it returns `stop_reason="tool_use"`. You execute the tool and send the result back.
- **`@beta_tool`** — A decorator that auto-generates tool schemas from Python functions (like OpenAI's `@function_tool`).
- **Tool Runner** — `client.beta.messages.tool_runner()` automates the tool-call loop for you.
- **Multi-agent** — You implement routing logic yourself: a "triage" agent decides which specialist to hand off to based on the conversation.

> **Docs:** https://docs.anthropic.com/en/docs/build-with-claude/tool-use/overview  
> **SDK:** https://github.com/anthropics/anthropic-sdk-python  
> **Tool helpers:** https://github.com/anthropics/anthropic-sdk-python/blob/main/tools.md

---
## 1 · Installation & Setup

In [ ]:
# Install the Anthropic SDK (only needs to run once)
%pip install anthropic --quiet

In [ ]:
import os

# Set your Anthropic API key
# Option A: set it here directly (not recommended for production)
# os.environ["ANTHROPIC_API_KEY"] = "sk-ant-..."

# Option B: load from a .env file (recommended)
# %pip install python-dotenv --quiet
# from dotenv import load_dotenv
# load_dotenv()

# Verify the key is set
assert os.environ.get("ANTHROPIC_API_KEY"), "⚠️  Please set ANTHROPIC_API_KEY before continuing."
print("✅ API key is set.")

---
## 2 · Your First Claude Message — Hello World

The **Messages API** is the core of everything. You send a list of messages and get a response.

Key parameters:
- **`model`** — Which Claude model to use (e.g. `claude-sonnet-4-20250514`).
- **`max_tokens`** — Maximum tokens in the response.
- **`system`** — System prompt (optional) — tells Claude how to behave.
- **`messages`** — The conversation history (`role`: `"user"` or `"assistant"`).

In [ ]:
from anthropic import Anthropic

client = Anthropic()

# Simple message — ask Claude to write a haiku
message = client.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=256,
    system="You are a creative poet. When the user asks for something, respond with a haiku.",
    messages=[
        {"role": "user", "content": "Write a haiku about Python programming."}
    ],
)

print("Response:", message.content[0].text)
print(f"\nTokens used — input: {message.usage.input_tokens}, output: {message.usage.output_tokens}")
print(f"Stop reason: {message.stop_reason}")

### Understanding the response

The `message` object contains:
- **`content`** — A list of content blocks (text, tool_use, etc.)
- **`stop_reason`** — Why Claude stopped: `"end_turn"` (done), `"tool_use"` (wants to call a tool), `"max_tokens"` (hit limit)
- **`usage`** — Token counts for billing
- **`model`** — The model used
- **`role`** — Always `"assistant"` for responses

In [ ]:
# Let's inspect the full response structure
print("Model:", message.model)
print("Role:", message.role)
print("Stop reason:", message.stop_reason)
print()
print("Content blocks:")
for i, block in enumerate(message.content):
    print(f"  [{i}] type={block.type}")
    if block.type == "text":
        print(f"       text={block.text}")

---
## 3 · Function Calling (Tool Use) — Manual Loop

This is the most powerful feature: you define **tools** that Claude can call.

How it works:
1. You define tools with a **name**, **description**, and **input_schema** (JSON Schema).
2. You send your request with `tools=[...]`.
3. Claude decides whether to call a tool. If yes, `stop_reason` is `"tool_use"`.
4. You extract the tool call, execute the function, and send the result back.
5. Claude uses the result to form its final answer.

### 3.1 — Defining tools and the agentic loop

In [ ]:
import json
from anthropic import Anthropic

client = Anthropic()

# ---------- Define tools as JSON schemas ----------

tools = [
    {
        "name": "get_weather",
        "description": "Get the current weather for a city. Returns temperature and conditions.",
        "input_schema": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The name of the city to check weather for.",
                }
            },
            "required": ["city"],
        },
    },
    {
        "name": "calculate",
        "description": "Evaluate a mathematical expression and return the result.",
        "input_schema": {
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string",
                    "description": "A mathematical expression to evaluate, e.g. '2 + 2' or '100 * 1.25'.",
                }
            },
            "required": ["expression"],
        },
    },
]


# ---------- Implement the actual tool functions ----------

def get_weather(city: str) -> str:
    """Fake weather lookup."""
    fake_weather = {
        "oslo": "Cloudy, 3°C, light rain",
        "paris": "Sunny, 18°C",
        "tokyo": "Humid, 28°C, partly cloudy",
        "new york": "Clear skies, 15°C",
    }
    return fake_weather.get(city.lower(), f"Weather data not available for {city}")


def calculate(expression: str) -> str:
    """Simple calculator."""
    try:
        result = eval(expression)  # demo only — don't use eval in production!
        return str(result)
    except Exception as e:
        return f"Error: {e}"


# Map tool names to functions
TOOL_FUNCTIONS = {
    "get_weather": get_weather,
    "calculate": calculate,
}

print("✅ Tools defined.")

In [ ]:
# ---------- The agentic tool-use loop ----------

def run_agent(user_message: str, system: str = "", tools: list = [], tool_functions: dict = {}, model: str = "claude-sonnet-4-20250514") -> str:
    """
    Run a single-turn agentic loop:
    1. Send user message + tools to Claude.
    2. If Claude wants to use a tool, execute it and send the result back.
    3. Repeat until Claude produces a final text response.
    """
    messages = [{"role": "user", "content": user_message}]

    while True:
        # Call Claude
        response = client.messages.create(
            model=model,
            max_tokens=1024,
            system=system,
            tools=tools,
            messages=messages,
        )

        print(f"  [stop_reason: {response.stop_reason}]")

        # If Claude is done talking, return the text
        if response.stop_reason == "end_turn":
            # Extract text from the response
            text_parts = [block.text for block in response.content if block.type == "text"]
            return "\n".join(text_parts)

        # If Claude wants to use tools
        if response.stop_reason == "tool_use":
            # Add Claude's response (with tool_use blocks) to messages
            messages.append({"role": "assistant", "content": response.content})

            # Process each tool call
            tool_results = []
            for block in response.content:
                if block.type == "tool_use":
                    tool_name = block.name
                    tool_input = block.input
                    tool_use_id = block.id

                    print(f"  🔧 Tool call: {tool_name}({json.dumps(tool_input)})")

                    # Execute the tool
                    if tool_name in tool_functions:
                        result = tool_functions[tool_name](**tool_input)
                    else:
                        result = f"Error: Unknown tool '{tool_name}'"

                    print(f"  📤 Result: {result}")

                    tool_results.append({
                        "type": "tool_result",
                        "tool_use_id": tool_use_id,
                        "content": result,
                    })

            # Send tool results back to Claude
            messages.append({"role": "user", "content": tool_results})

        else:
            # Unexpected stop reason
            return f"Unexpected stop reason: {response.stop_reason}"


# Try it out!
print("=" * 60)
print("Asking Claude about weather and math...")
print("=" * 60)

result = run_agent(
    user_message="What's the weather in Oslo? Also, what is 42 * 17?",
    system="You are a helpful assistant. Use the tools available to you to answer the user's questions. Always prefer using a tool over making up information.",
    tools=tools,
    tool_functions=TOOL_FUNCTIONS,
)

print("\n💬 Final response:")
print(result)

### 3.2 — What happened under the hood?

The loop above is the **core pattern** for all agentic workflows with Claude:

```
User message  ──►  Claude  ──►  stop_reason="tool_use"  ──►  Execute tool(s)
                     ▲                                            │
                     └────────────  tool_result  ◄────────────────┘
                                                        
                   Claude  ──►  stop_reason="end_turn"  ──►  Final text response
```

Key points:
- Claude can call **multiple tools in parallel** in a single response.
- The loop continues until `stop_reason == "end_turn"`.
- Each tool result must reference the `tool_use_id` from the corresponding tool call.
- The full conversation history (including tool calls & results) is maintained in `messages`.

---
## 4 · Function Calling with `@beta_tool` + Tool Runner (Automatic Loop)

The manual loop above is great for understanding, but the SDK provides a **much simpler** way:

1. **`@beta_tool`** — Decorator that auto-generates the tool schema from your Python function's signature + docstring.
2. **`client.beta.messages.tool_runner()`** — Runs the entire tool-use loop for you automatically.

This is the recommended approach for most use cases.

### 4.1 — Using `@beta_tool`

In [ ]:
from anthropic import Anthropic, beta_tool

client = Anthropic()


@beta_tool
def get_weather(city: str) -> str:
    """Get the current weather for a city.

    Args:
        city: The name of the city to check weather for.
    """
    fake_weather = {
        "oslo": "Cloudy, 3°C, light rain",
        "paris": "Sunny, 18°C",
        "tokyo": "Humid, 28°C, partly cloudy",
        "new york": "Clear skies, 15°C",
    }
    return fake_weather.get(city.lower(), f"Weather data not available for {city}")


@beta_tool
def calculate(expression: str) -> str:
    """Evaluate a mathematical expression and return the result.

    Args:
        expression: A mathematical expression, e.g. '2 + 2' or '100 * 1.25'.
    """
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Error: {e}"


# Let's see the auto-generated schema
print("Auto-generated tool schema for get_weather:")
print(json.dumps(get_weather.to_dict(), indent=2))

### 4.2 — Using the Tool Runner

The **tool runner** handles the entire loop for you. Each iteration of the runner yields a `Message` object. It stops when Claude produces a final response (no more tool calls).

In [ ]:
# The tool runner handles the entire loop automatically!
runner = client.beta.messages.tool_runner(
    model="claude-sonnet-4-20250514",
    max_tokens=1024,
    system="You are a helpful assistant. Use the tools available to answer questions accurately.",
    tools=[get_weather, calculate],
    messages=[{"role": "user", "content": "What's the weather in Tokyo? And what is 256 / 8?"}],
)

# Each iteration yields a message from Claude
print("=== Tool Runner Iterations ===")
for i, message in enumerate(runner, 1):
    print(f"\n--- Iteration {i} (stop_reason: {message.stop_reason}) ---")
    for block in message.content:
        if block.type == "text":
            print(f"💬 Text: {block.text}")
        elif block.type == "tool_use":
            print(f"🔧 Tool call: {block.name}({json.dumps(block.input)})")

# The last message has the final response
print("\n" + "=" * 60)
print("✅ Final response:")
final_text = [b.text for b in message.content if b.type == "text"]
print("\n".join(final_text))

### 4.3 — Tools with complex types

You can use richer schemas for more structured tool inputs:

In [ ]:
# --- Sample data (pretend this is a database) ---
ORDERS_DB = {
    "ORD-001": {"status": "shipped", "item": "Laptop", "tracking": "TRACK-9876", "total": 1299.99},
    "ORD-002": {"status": "processing", "item": "Headphones", "tracking": None, "total": 249.50},
    "ORD-003": {"status": "delivered", "item": "Keyboard", "tracking": "TRACK-5432", "total": 89.99},
}


@beta_tool
def lookup_order(order_id: str, include_tracking: bool = False) -> str:
    """Look up an order by its ID.

    Args:
        order_id: The order ID (e.g. ORD-001).
        include_tracking: Whether to include tracking information.
    """
    order = ORDERS_DB.get(order_id)
    if not order:
        return f"Order {order_id} not found."

    info = f"Order {order_id}: {order['item']} — Status: {order['status']} — Total: ${order['total']}"
    if include_tracking and order["tracking"]:
        info += f" — Tracking: {order['tracking']}"
    return info


@beta_tool
def list_all_orders() -> str:
    """List all available orders in the system."""
    lines = []
    for oid, data in ORDERS_DB.items():
        lines.append(f"{oid}: {data['item']} ({data['status']})")
    return "\n".join(lines)


# Run with the tool runner
runner = client.beta.messages.tool_runner(
    model="claude-sonnet-4-20250514",
    max_tokens=1024,
    system="You help users check their order status. Use the tools to look up orders.",
    tools=[lookup_order, list_all_orders],
    messages=[{"role": "user", "content": "Show me all my orders, then give me details on ORD-001 with tracking."}],
)

for message in runner:
    pass  # let the runner iterate

final_text = [b.text for b in message.content if b.type == "text"]
print("\n".join(final_text))

---
## 5 · Multi-Agent System — Routing & Handoffs

Claude's API doesn't have a built-in "handoff" primitive like OpenAI's Agents SDK. Instead, you implement **multi-agent routing** yourself — which gives you full control.

The pattern:
1. Define **specialist agents** — each with their own system prompt and tools.
2. Create a **triage agent** that decides which specialist to route to.
3. The triage agent uses a "route" tool to hand off to the right specialist.
4. The specialist handles the conversation with its own tools.

### 5.1 — Defining specialist agents as classes

In [ ]:
from anthropic import Anthropic, beta_tool
import json

client = Anthropic()
MODEL = "claude-sonnet-4-20250514"


class Agent:
    """A simple agent wrapper around Claude with a system prompt and tools."""

    def __init__(self, name: str, system: str, tools: list = None):
        self.name = name
        self.system = system
        self.tools = tools or []
        # Build a map from tool name → callable for execution
        self._tool_map = {}
        for tool in self.tools:
            if hasattr(tool, "name"):
                self._tool_map[tool.name] = tool

    def run(self, user_message: str) -> dict:
        """
        Run this agent with a user message.
        Uses the tool runner if tools are defined, otherwise a simple message call.
        Returns {"agent": name, "response": text, "messages": conversation_history}.
        """
        if self.tools:
            # Use the tool runner for automatic tool execution
            runner = client.beta.messages.tool_runner(
                model=MODEL,
                max_tokens=1024,
                system=self.system,
                tools=self.tools,
                messages=[{"role": "user", "content": user_message}],
            )
            for message in runner:
                pass  # iterate to completion
        else:
            message = client.messages.create(
                model=MODEL,
                max_tokens=1024,
                system=self.system,
                messages=[{"role": "user", "content": user_message}],
            )

        final_text = [b.text for b in message.content if b.type == "text"]
        return {
            "agent": self.name,
            "response": "\n".join(final_text),
        }


print("✅ Agent class defined.")

### 5.2 — Triage pattern (one router → specialized agents)

We'll create a **triage agent** that uses a `route_to_agent` tool to decide which specialist should handle the request. Then we run the selected specialist.

In [ ]:
# --- Define specialist agents ---

billing_agent = Agent(
    name="Billing Agent",
    system=(
        "You are a billing specialist. You help users with invoices, payments, and billing questions. "
        "If the user asks about something outside billing, tell them you can only help with billing."
    ),
)

tech_support_agent = Agent(
    name="Tech Support Agent",
    system=(
        "You are a tech support specialist. You help users with technical issues, bugs, and setup problems. "
        "If the user asks about something outside tech support, tell them you can only help with technical issues."
    ),
)

sales_agent = Agent(
    name="Sales Agent",
    system=(
        "You are a sales specialist. You help users with product information, pricing, and purchases. "
        "If the user asks about something outside sales, tell them you can only help with sales."
    ),
)

# Registry of specialist agents
AGENTS = {
    "billing": billing_agent,
    "tech_support": tech_support_agent,
    "sales": sales_agent,
}

print("✅ Specialist agents defined.")
print(f"   Available agents: {list(AGENTS.keys())}")

In [ ]:
def triage_and_route(user_message: str) -> dict:
    """
    Triage a user message and route to the appropriate specialist agent.
    
    Step 1: Ask Claude (as a triage agent) to decide which specialist to route to.
    Step 2: Run the selected specialist agent with the original user message.
    """
    
    # Define the routing tool
    route_tool = {
        "name": "route_to_agent",
        "description": (
            "Route the customer to the appropriate specialist agent. "
            "Call this tool once you understand what the customer needs."
        ),
        "input_schema": {
            "type": "object",
            "properties": {
                "agent": {
                    "type": "string",
                    "enum": ["billing", "tech_support", "sales"],
                    "description": "The specialist agent to route to.",
                },
                "reason": {
                    "type": "string",
                    "description": "Brief reason for the routing decision.",
                },
            },
            "required": ["agent", "reason"],
        },
    }

    # Step 1: Triage — ask Claude to decide where to route
    triage_response = client.messages.create(
        model=MODEL,
        max_tokens=256,
        system=(
            "You are a customer support triage agent. Your ONLY job is to understand the user's request "
            "and route them to the correct specialist by calling the route_to_agent tool.\n\n"
            "Route to:\n"
            "- billing: for invoices, payments, charges, account balance\n"
            "- tech_support: for technical issues, bugs, errors, setup problems\n"
            "- sales: for product info, pricing, purchasing, upgrades\n\n"
            "Always call the route_to_agent tool. Do not answer the question yourself."
        ),
        tools=[route_tool],
        tool_choice={"type": "tool", "name": "route_to_agent"},  # Force tool use
        messages=[{"role": "user", "content": user_message}],
    )

    # Extract the routing decision
    for block in triage_response.content:
        if block.type == "tool_use" and block.name == "route_to_agent":
            agent_key = block.input["agent"]
            reason = block.input["reason"]
            break
    else:
        return {"error": "Triage agent did not route to any specialist."}

    print(f"🔀 Triage decision: route to '{agent_key}' — {reason}")

    # Step 2: Run the selected specialist
    specialist = AGENTS[agent_key]
    result = specialist.run(user_message)

    return result


# Test: billing question
print("=" * 60)
print("TEST 1: Billing question")
print("=" * 60)
result = triage_and_route("I was charged twice for my last invoice. Can you help?")
print(f"\n🤖 Handled by: {result['agent']}")
print(f"💬 Response: {result['response']}")

In [ ]:
# Test: tech support question
print("=" * 60)
print("TEST 2: Tech support question")
print("=" * 60)
result2 = triage_and_route("My app keeps crashing when I try to upload a file.")
print(f"\n🤖 Handled by: {result2['agent']}")
print(f"💬 Response: {result2['response']}")

In [ ]:
# Test: sales question
print("=" * 60)
print("TEST 3: Sales question")
print("=" * 60)
result3 = triage_and_route("What are your pricing plans for the enterprise tier?")
print(f"\n🤖 Handled by: {result3['agent']}")
print(f"💬 Response: {result3['response']}")

---
## 6 · Full Practical Example — Customer Support System

Let's combine everything: **tools + multi-agent routing** into a realistic customer support scenario with sample data.

### The scenario
A company has:
- A **triage agent** that routes customers.
- An **order agent** with tools to look up orders and process returns.
- A **billing agent** with tools to check account balance and apply credits.
- An **FAQ agent** for general questions.

In [ ]:
from anthropic import Anthropic, beta_tool
import json

client = Anthropic()
MODEL = "claude-sonnet-4-20250514"

# ============================
# SAMPLE DATA (our "database")
# ============================

CUSTOMERS = {
    "CUST-100": {"name": "Alice Johnson", "email": "alice@example.com", "balance": 150.00},
    "CUST-200": {"name": "Bob Smith", "email": "bob@example.com", "balance": 0.00},
    "CUST-300": {"name": "Carol Williams", "email": "carol@example.com", "balance": 45.50},
}

ORDERS = {
    "ORD-1001": {"customer": "CUST-100", "item": "Wireless Mouse", "price": 29.99, "status": "delivered"},
    "ORD-1002": {"customer": "CUST-100", "item": "USB-C Hub", "price": 49.99, "status": "shipped"},
    "ORD-1003": {"customer": "CUST-200", "item": "Mechanical Keyboard", "price": 89.99, "status": "processing"},
    "ORD-1004": {"customer": "CUST-300", "item": "Monitor Stand", "price": 45.50, "status": "delivered"},
}

FAQ_DATA = {
    "shipping": "We offer free shipping on orders over $50. Standard shipping takes 5-7 business days.",
    "returns": "You can return items within 30 days of delivery for a full refund.",
    "warranty": "All products come with a 1-year manufacturer warranty.",
    "hours": "Our support team is available Monday-Friday, 9 AM - 6 PM EST.",
}

print("✅ Sample data loaded.")

In [ ]:
# ============================
# TOOLS for the Order Agent
# ============================

@beta_tool
def get_order_status(order_id: str) -> str:
    """Look up the status of an order.

    Args:
        order_id: The order ID, e.g. ORD-1001.
    """
    order = ORDERS.get(order_id)
    if not order:
        return f"Order {order_id} not found."
    return json.dumps(order, indent=2)


@beta_tool
def get_customer_orders(customer_id: str) -> str:
    """Get all orders for a customer.

    Args:
        customer_id: The customer ID, e.g. CUST-100.
    """
    customer_orders = {oid: o for oid, o in ORDERS.items() if o["customer"] == customer_id}
    if not customer_orders:
        return f"No orders found for customer {customer_id}."
    return json.dumps(customer_orders, indent=2)


@beta_tool
def initiate_return(order_id: str, reason: str) -> str:
    """Initiate a return for an order.

    Args:
        order_id: The order ID to return.
        reason: The reason for the return.
    """
    order = ORDERS.get(order_id)
    if not order:
        return f"Order {order_id} not found."
    if order["status"] != "delivered":
        return f"Cannot return order {order_id} — current status is '{order['status']}'. Only delivered orders can be returned."
    return f"✅ Return initiated for {order_id} ({order['item']}). Reason: {reason}. Refund of ${order['price']} will be processed within 5-7 business days."


# ============================
# TOOLS for the Billing Agent
# ============================

@beta_tool
def get_account_balance(customer_id: str) -> str:
    """Check the account balance/credits for a customer.

    Args:
        customer_id: The customer ID.
    """
    customer = CUSTOMERS.get(customer_id)
    if not customer:
        return f"Customer {customer_id} not found."
    return f"Customer {customer['name']} ({customer_id}) has a balance of ${customer['balance']:.2f}"


@beta_tool
def apply_credit(customer_id: str, amount: float, reason: str) -> str:
    """Apply a credit to a customer's account.

    Args:
        customer_id: The customer ID.
        amount: The credit amount to apply (positive number).
        reason: The reason for the credit.
    """
    customer = CUSTOMERS.get(customer_id)
    if not customer:
        return f"Customer {customer_id} not found."
    if amount <= 0:
        return "Credit amount must be positive."
    new_balance = customer["balance"] + amount
    return f"✅ Credit of ${amount:.2f} applied to {customer['name']}'s account. Reason: {reason}. New balance: ${new_balance:.2f}"


# ============================
# TOOL for the FAQ Agent
# ============================

@beta_tool
def search_faq(topic: str) -> str:
    """Search the FAQ database for information on a topic.

    Args:
        topic: The topic to search for (e.g. 'shipping', 'returns', 'warranty').
    """
    results = []
    for key, value in FAQ_DATA.items():
        if topic.lower() in key or key in topic.lower():
            results.append(f"{key.upper()}: {value}")
    if not results:
        return f"No FAQ entries found for '{topic}'. Available topics: {', '.join(FAQ_DATA.keys())}"
    return "\n".join(results)


print("✅ All tools defined.")

In [ ]:
# ============================
# BUILD THE MULTI-AGENT SYSTEM
# ============================

# Specialist agents with their tools
SPECIALIST_AGENTS = {
    "order": {
        "system": (
            "You are an order management specialist. Help customers check order status, "
            "view their orders, and process returns. Use your tools to look up information. "
            "The current customer is Alice Johnson (CUST-100). "
            "Always be helpful and professional."
        ),
        "tools": [get_order_status, get_customer_orders, initiate_return],
    },
    "billing": {
        "system": (
            "You are a billing specialist. Help customers with account balance inquiries "
            "and apply credits when appropriate. Use your tools to look up information. "
            "The current customer is Alice Johnson (CUST-100). "
            "Always be helpful and professional."
        ),
        "tools": [get_account_balance, apply_credit],
    },
    "faq": {
        "system": (
            "You are an FAQ specialist. Answer general questions about the company's policies. "
            "Use the FAQ search tool to find accurate information. Be friendly and concise."
        ),
        "tools": [search_faq],
    },
}


def run_customer_support(user_message: str) -> dict:
    """
    Full customer support pipeline:
    1. Triage agent decides which specialist to route to.
    2. Specialist agent handles the request with its tools.
    """
    
    # --- Step 1: Triage ---
    route_tool = {
        "name": "route_to_agent",
        "description": "Route the customer to the appropriate specialist agent.",
        "input_schema": {
            "type": "object",
            "properties": {
                "agent": {
                    "type": "string",
                    "enum": ["order", "billing", "faq"],
                    "description": "The specialist: 'order' for orders/returns, 'billing' for balance/credits, 'faq' for general questions.",
                },
                "reason": {
                    "type": "string",
                    "description": "Brief reason for the routing decision.",
                },
            },
            "required": ["agent", "reason"],
        },
    }

    triage_response = client.messages.create(
        model=MODEL,
        max_tokens=256,
        system=(
            "You are a customer support triage agent. Route the customer to the correct specialist "
            "by calling the route_to_agent tool.\n"
            "- order: order status, returns, deliveries\n"
            "- billing: account balance, credits, payments\n"
            "- faq: general questions, policies, shipping info"
        ),
        tools=[route_tool],
        tool_choice={"type": "tool", "name": "route_to_agent"},
        messages=[{"role": "user", "content": user_message}],
    )

    # Extract routing decision
    agent_key = None
    reason = ""
    for block in triage_response.content:
        if block.type == "tool_use":
            agent_key = block.input["agent"]
            reason = block.input["reason"]
            break

    if not agent_key:
        return {"error": "Triage failed"}

    print(f"🔀 Triage: routed to '{agent_key}' — {reason}")

    # --- Step 2: Run the specialist ---
    specialist = SPECIALIST_AGENTS[agent_key]

    runner = client.beta.messages.tool_runner(
        model=MODEL,
        max_tokens=1024,
        system=specialist["system"],
        tools=specialist["tools"],
        messages=[{"role": "user", "content": user_message}],
    )

    # Collect tool calls for tracing
    trace = []
    for message in runner:
        for block in message.content:
            if block.type == "tool_use":
                trace.append(f"🔧 {block.name}({json.dumps(block.input)})")

    final_text = [b.text for b in message.content if b.type == "text"]

    return {
        "agent": agent_key,
        "response": "\n".join(final_text),
        "tool_calls": trace,
    }


print("✅ Multi-agent customer support system built.")

In [ ]:
# --- Test 1: Order-related query ---
print("=" * 60)
print("TEST 1: Order query")
print("=" * 60)

result = run_customer_support(
    "Hi, I'd like to check the status of my orders and maybe return the wireless mouse."
)

print(f"\n🤖 Handled by: {result['agent']}")
print(f"🔧 Tool calls: {result['tool_calls']}")
print(f"\n💬 Response:\n{result['response']}")

In [ ]:
# --- Test 2: Billing query ---
print("=" * 60)
print("TEST 2: Billing query")
print("=" * 60)

result2 = run_customer_support(
    "What's my current account balance? I'm customer CUST-100."
)

print(f"\n🤖 Handled by: {result2['agent']}")
print(f"🔧 Tool calls: {result2['tool_calls']}")
print(f"\n💬 Response:\n{result2['response']}")

In [ ]:
# --- Test 3: FAQ query ---
print("=" * 60)
print("TEST 3: FAQ query")
print("=" * 60)

result3 = run_customer_support(
    "What is your return policy and shipping times?"
)

print(f"\n🤖 Handled by: {result3['agent']}")
print(f"🔧 Tool calls: {result3['tool_calls']}")
print(f"\n💬 Response:\n{result3['response']}")

---
## 7 · Inspecting Results & Conversation History

Understanding what happened during a run is crucial for debugging. Let's write a version that captures the full conversation trace, including every tool call and result.

In [ ]:
def run_agent_with_trace(user_message: str, system: str, tools: list, model: str = MODEL) -> dict:
    """
    Run an agent loop manually and capture a full trace of the conversation.
    Returns the final response and a detailed trace of everything that happened.
    """
    messages = [{"role": "user", "content": user_message}]
    trace = []
    iteration = 0
    
    # Build tool map for execution
    tool_map = {}
    tool_defs = []
    for t in tools:
        if hasattr(t, "to_dict"):
            tool_defs.append(t.to_dict())
            tool_map[t.name] = t
        else:
            tool_defs.append(t)

    while True:
        iteration += 1
        
        response = client.messages.create(
            model=model,
            max_tokens=1024,
            system=system,
            tools=tool_defs,
            messages=messages,
        )

        trace.append({
            "iteration": iteration,
            "stop_reason": response.stop_reason,
            "usage": {"input": response.usage.input_tokens, "output": response.usage.output_tokens},
            "content": [],
        })

        for block in response.content:
            if block.type == "text":
                trace[-1]["content"].append({"type": "text", "text": block.text})
            elif block.type == "tool_use":
                trace[-1]["content"].append({
                    "type": "tool_call",
                    "name": block.name,
                    "input": block.input,
                    "id": block.id,
                })

        if response.stop_reason == "end_turn":
            final_text = [b.text for b in response.content if b.type == "text"]
            return {
                "response": "\n".join(final_text),
                "trace": trace,
                "iterations": iteration,
            }

        if response.stop_reason == "tool_use":
            messages.append({"role": "assistant", "content": response.content})

            tool_results = []
            for block in response.content:
                if block.type == "tool_use":
                    if block.name in tool_map:
                        result = tool_map[block.name](**block.input)
                    else:
                        result = f"Unknown tool: {block.name}"

                    trace[-1]["content"].append({
                        "type": "tool_result",
                        "name": block.name,
                        "result": result,
                    })

                    tool_results.append({
                        "type": "tool_result",
                        "tool_use_id": block.id,
                        "content": result,
                    })

            messages.append({"role": "user", "content": tool_results})
        else:
            return {"response": f"Unexpected stop_reason: {response.stop_reason}", "trace": trace, "iterations": iteration}


def print_trace(result: dict):
    """Pretty-print a conversation trace."""
    print("\n" + "=" * 60)
    print("📋 CONVERSATION TRACE")
    print("=" * 60)

    for step in result["trace"]:
        print(f"\n--- Iteration {step['iteration']} (stop_reason: {step['stop_reason']}) ---")
        print(f"    Tokens: input={step['usage']['input']}, output={step['usage']['output']}")

        for item in step["content"]:
            if item["type"] == "text":
                print(f"    💬 Text: {item['text'][:200]}")
            elif item["type"] == "tool_call":
                print(f"    🔧 Tool call: {item['name']}({json.dumps(item['input'])})")
            elif item["type"] == "tool_result":
                print(f"    📤 Tool result ({item['name']}): {item['result'][:200]}")

    print(f"\n{'=' * 60}")
    print(f"✅ Completed in {result['iterations']} iteration(s)")
    print(f"✅ Final response: {result['response'][:300]}")
    print("=" * 60)

In [ ]:
# Run with full tracing
traced_result = run_agent_with_trace(
    user_message="Show me all orders for customer CUST-100 and then return the wireless mouse.",
    system=(
        "You are an order management specialist. Help customers check order status, "
        "view their orders, and process returns. The current customer is Alice Johnson (CUST-100)."
    ),
    tools=[get_order_status, get_customer_orders, initiate_return],
)

print_trace(traced_result)

### Understanding the trace

The trace shows exactly what happened at each step:

| Step | What happens |
|---|---|
| **Iteration 1** | Claude receives the user message, decides to call tools (e.g., `get_customer_orders`). `stop_reason="tool_use"`. |
| **Iteration 2** | Claude receives the tool result, may call more tools (e.g., `initiate_return`). |
| **Iteration N** | Claude has all the info it needs, produces a final text response. `stop_reason="end_turn"`. |

Key observations:
- **Input tokens grow** each iteration because the full conversation history is sent.
- Claude can call **multiple tools in parallel** in a single iteration.
- The `tool_use_id` links each result to the correct tool call.

---
## 8 · Proposed Project Structure for Production

When moving beyond a notebook into a real project, here's how you should structure a multi-agent system:

```
my-agent-project/
│
├── .env                          # API keys (ANTHROPIC_API_KEY, etc.) — never commit this!
├── .env.example                  # Template showing required env vars
├── .gitignore                    # Ignore .env, __pycache__, .venv, etc.
├── pyproject.toml                # Project metadata & dependencies
├── README.md                     # Project documentation
│
├── src/
│   ├── __init__.py
│   │
│   ├── agents/                   # Agent definitions — one file per agent (or group)
│   │   ├── __init__.py
│   │   ├── base.py               # Base Agent class with run() and tracing
│   │   ├── triage.py             # Triage / router agent
│   │   ├── order_agent.py        # Order management specialist
│   │   ├── billing_agent.py      # Billing specialist
│   │   └── faq_agent.py          # FAQ / general questions
│   │
│   ├── tools/                    # Tool functions — grouped by domain
│   │   ├── __init__.py
│   │   ├── order_tools.py        # get_order_status, initiate_return, etc.
│   │   ├── billing_tools.py      # get_account_balance, apply_credit, etc.
│   │   └── faq_tools.py          # search_faq, etc.
│   │
│   ├── models/                   # Pydantic models for structured data
│   │   ├── __init__.py
│   │   ├── customer.py           # Customer data models
│   │   └── order.py              # Order, ReturnRequest, etc.
│   │
│   ├── services/                 # External integrations (DB, APIs, etc.)
│   │   ├── __init__.py
│   │   ├── database.py           # Database connection and queries
│   │   └── external_api.py       # External API clients
│   │
│   ├── config.py                 # App configuration, load .env, model settings
│   └── main.py                   # Entry point — build the agent graph and run
│
├── notebooks/                    # Jupyter notebooks for experimentation
│   └── exploration.ipynb
│
└── tests/                        # Tests for tools, agents, and flows
    ├── __init__.py
    ├── test_tools.py             # Unit tests for tool functions
    ├── test_agents.py            # Integration tests for agent behavior
    └── test_flows.py             # End-to-end conversation flow tests
```

### Key design principles

| Principle | Why |
|---|---|
| **Separate agents from tools** | Agents are *configuration* (system prompt, model, tools list). Tools are *logic*. Keep them apart so you can test tools independently. |
| **One agent per file** | Makes it easy to find, update, and version-control each agent's prompt and behavior. |
| **Group tools by domain** | `order_tools.py`, `billing_tools.py` — each file contains related tools. Easier to maintain. |
| **Pydantic models in `models/`** | Define your data shapes once, reuse in tools and agents. |
| **Services layer** | Real database queries and API calls go here. Tools call services, not the other way around. |
| **Config in one place** | Load `.env`, set model names, max_tokens, etc. in `config.py`. |
| **Test your tools** | Tool functions are pure Python — easy to unit test. Test that `get_order_status("ORD-001")` returns the right thing. |

### Example: what `src/agents/base.py` would look like

```python
# src/agents/base.py
from anthropic import Anthropic
from src.config import MODEL, MAX_TOKENS

client = Anthropic()


class Agent:
    """Base agent class wrapping Claude with system prompt and tools."""

    def __init__(self, name: str, system: str, tools: list = None):
        self.name = name
        self.system = system
        self.tools = tools or []

    def run(self, user_message: str) -> dict:
        """Run the agent with automatic tool execution via tool_runner."""
        if self.tools:
            runner = client.beta.messages.tool_runner(
                model=MODEL,
                max_tokens=MAX_TOKENS,
                system=self.system,
                tools=self.tools,
                messages=[{"role": "user", "content": user_message}],
            )
            for message in runner:
                pass
        else:
            message = client.messages.create(
                model=MODEL,
                max_tokens=MAX_TOKENS,
                system=self.system,
                messages=[{"role": "user", "content": user_message}],
            )

        text = [b.text for b in message.content if b.type == "text"]
        return {"agent": self.name, "response": "\n".join(text)}
```

### Example: what `src/tools/order_tools.py` would look like

```python
# src/tools/order_tools.py
from anthropic import beta_tool
from src.services.database import get_order, get_orders_for_customer


@beta_tool
def get_order_status(order_id: str) -> str:
    """Look up the status of an order.

    Args:
        order_id: The order ID, e.g. ORD-1001.
    """
    order = get_order(order_id)
    if not order:
        return f"Order {order_id} not found."
    return order.to_summary_string()
```

### Example: what `src/agents/triage.py` would look like

```python
# src/agents/triage.py
from anthropic import Anthropic
from src.agents.order_agent import order_agent
from src.agents.billing_agent import billing_agent
from src.agents.faq_agent import faq_agent
from src.config import MODEL

client = Anthropic()

SPECIALISTS = {
    "order": order_agent,
    "billing": billing_agent,
    "faq": faq_agent,
}

ROUTE_TOOL = {
    "name": "route_to_agent",
    "description": "Route the customer to the appropriate specialist.",
    "input_schema": {
        "type": "object",
        "properties": {
            "agent": {
                "type": "string",
                "enum": list(SPECIALISTS.keys()),
            },
            "reason": {"type": "string"},
        },
        "required": ["agent", "reason"],
    },
}


def triage_and_route(user_message: str) -> dict:
    response = client.messages.create(
        model=MODEL,
        max_tokens=256,
        system="You are a triage agent. Route customers to the right specialist.",
        tools=[ROUTE_TOOL],
        tool_choice={"type": "tool", "name": "route_to_agent"},
        messages=[{"role": "user", "content": user_message}],
    )

    for block in response.content:
        if block.type == "tool_use":
            specialist = SPECIALISTS[block.input["agent"]]
            return specialist.run(user_message)

    return {"error": "Triage failed"}
```

### Example: what `src/main.py` would look like

```python
# src/main.py
from src.agents.triage import triage_and_route
from src.config import load_config


def main():
    load_config()  # loads .env, validates API key

    result = triage_and_route(
        "I'd like to check on my order ORD-1001."
    )
    print(f"Agent: {result['agent']}")
    print(f"Response: {result['response']}")


if __name__ == "__main__":
    main()
```

---

## Summary & Cheatsheet

| What | How |
|---|---|
| Create a client | `client = Anthropic()` |
| Send a message | `client.messages.create(model=..., max_tokens=..., messages=[...])` |
| Set system prompt | `system="You are a helpful assistant."` parameter |
| Define a tool (manual) | Dict with `name`, `description`, `input_schema` (JSON Schema) |
| Define a tool (auto) | `@beta_tool` decorator on a Python function |
| Auto tool schema | `my_tool.to_dict()` — see the generated schema |
| Run tool loop (manual) | Check `stop_reason == "tool_use"`, execute, send `tool_result` back |
| Run tool loop (auto) | `client.beta.messages.tool_runner(tools=[...], ...)` |
| Force a specific tool | `tool_choice={"type": "tool", "name": "tool_name"}` |
| Force any tool | `tool_choice={"type": "any"}` |
| Let Claude decide | `tool_choice={"type": "auto"}` (default) |
| Multi-agent routing | Use a routing tool + `tool_choice` to pick a specialist |
| Get text response | `[b.text for b in message.content if b.type == "text"]` |
| Check stop reason | `message.stop_reason` — `"end_turn"`, `"tool_use"`, `"max_tokens"` |
| Token usage | `message.usage.input_tokens`, `message.usage.output_tokens` |

### Key differences from OpenAI Agents SDK

| Concept | OpenAI Agents SDK | Anthropic SDK |
|---|---|---|
| Agent definition | `Agent(name, instructions, tools, handoffs)` | You define system prompt + tools; wrap in your own class |
| Tool definition | `@function_tool` | `@beta_tool` (auto) or manual JSON Schema dict |
| Agent loop | `Runner.run_sync(agent, msg)` | `client.beta.messages.tool_runner(...)` or manual loop |
| Handoffs | Built-in `handoffs=[agent_a, agent_b]` | Implement yourself via routing tool |
| Agents as tools | `agent.as_tool(...)` | Implement yourself (call sub-agent, return result) |
| Conversation state | Managed by the Runner | You manage the `messages` list yourself |

### Next steps
- 📖 [Anthropic API Docs](https://docs.anthropic.com/en/docs/build-with-claude/tool-use/overview)
- 🧰 [SDK Tool Helpers](https://github.com/anthropics/anthropic-sdk-python/blob/main/tools.md)
- 🔍 [Prompt Caching](https://docs.anthropic.com/en/docs/build-with-claude/prompt-caching) — reduce costs in multi-turn conversations
- 🌐 [MCP Integration](https://docs.anthropic.com/en/docs/agents-and-tools/mcp-connector) — connect to MCP servers
- 📊 [Structured Outputs](https://docs.anthropic.com/en/docs/build-with-claude/structured-outputs) — guarantee JSON schema conformance
- 🖥️ [Computer Use](https://docs.anthropic.com/en/docs/agents-and-tools/tool-use/computer-use-tool) — let Claude interact with a desktop